In [70]:
# import the necessary libraries
from bs4 import BeautifulSoup
import requests
import json
import re
import datetime
import easyocr
import cv2
import geopy.distance

In [27]:
# this function handles the situation when the html is embedded within javascript
def javascript_html_parse(source_arg):
	"""Return the JSON object as a dictionary"""
	start = "<script>window.__PRELOADED_STATE__ = "
	end = "</script"
	x = source_arg[source_arg.find(start)+len(start):]
	x = x[:x.find(end)]
	x = json.loads(x)
	return x

In [28]:
# Obtain all of the URL's that need to be scraped from 
# This is the first URL we start at, here we are collecting all of the URL's to be scraped from into 
# local area URL's 
url = "https://www.rightmove.co.uk/house-prices-in-London.html"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
b = soup.find_all("a", class_="head")
london_urls = []
for i in b:
	london_urls.append(i["href"])

borough_urls = []
for url in london_urls:
	source = requests.get(url).text
	soup = BeautifulSoup(source, 'lxml')
	b = soup.find_all("a", class_="head")
	for i in b:
		borough_urls.append(i["href"])

local_area_urls = []
for url in borough_urls:
	source = requests.get(url).text
	soup = BeautifulSoup(source, 'lxml')
	b = soup.find_all("a", class_="head")
	for i in b:
		local_area_urls.append(i["href"])	

In [29]:
# iterate over all of the URL's and scrape from them, and store the scraped data into a variable called houses
m_values = ["square meters", "square metres", "square meter","square metre","square m","square mt","sqmt","sq mt","sq.mt", "sq. mt", "sqm", "sq m", "sq.m", "sq. m", "sq: m", "sq:m", "sq :m", "sq:  m", "sq : m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
f_values = ['square feet','square ft','square f','sqft','sq ft','sq.ft','sq. ft','sqf','sq f','sq.f','sq. f','sq: m','sq:m','sq :m','sq : m','sq :m','feet2','ft2','f2']
houses = {}
for area_url in local_area_urls[:2]:

	# while loop that obtains the address, property type, last sold price, last sold date, and its url (if it has one) from each house. 
	# Stores each value into a list. This is from the web page which shows different houses from a particular area.

	page_number = 1
	addresses, property_type, last_sold_price, last_sold_date, url = [], [], [], [], []
	# obtain the number of pages to use in the while loop condition
	source = requests.get(area_url).text
	a = javascript_html_parse(source)
	num_pages = a["pagination"]["last"]
    
	while page_number <= num_pages:

		if page_number == 1:
			source = requests.get(area_url).text
		else:
			source = requests.get(area_url+f"?page={page_number}").text

		# Scrape the address, property type, the price & date of the last sale, and the URL.
		x = javascript_html_parse(source)
		for house in x["results"]["properties"]:
			addresses.append(house["address"])
			property_type.append(house["propertyType"])
			last_sold_price.append(house["transactions"][0]["displayPrice"])
			last_sold_date.append(house["transactions"][0]["dateSold"])
			url.append(house["detailUrl"])



		page_number += 1


	# Store the scraped data for each property into the houses dictionary
	for i in range(len(addresses)):
		houses[addresses[i]] = {"property_type":property_type[i], "price":last_sold_price[i], "date":last_sold_date[i], "url": url[i]}


    # Now loop through all of the stored URL's for each property, and scrape more data from it.

	for v in houses.values():
		if v['url'] == '':
			continue
		else:
			source = requests.get(v['url']).text
			soup = BeautifulSoup(source, 'lxml')
			script = soup.find("script", {"type":"text/javascript"}).text
			# Regex used to convert the JSON data structure into a python dict 
			# Use regex to extract json data from the script text
			json_script = re.findall(("(?s)(?<=window.PAGE_MODEL = )(.*$)"), script)[0]
			# Transforming json data within string into dictionary
			json_dict = json.loads(json_script)

			# The bullet pointed features 
			try:
				features = json_dict["soldPropertyData"]["property"]["keyFeatures"]
			except TypeError: 
				pass
			else:
				v["features"] = features
			# The floorplan URL
			try:
				floorplan_url = json_dict["soldPropertyData"]["property"]["floorplans"][0]['url']
			except Exception:
				# seen a TypeError and an IndexError so I'm going to capture all exceptions raised
				pass
			else:
				v["floorplan_url"] = floorplan_url	
			# Scrape the pictures of the house ### commented this out for now, as I'm not using the image data
			#house_image_urls = []
			#for image in json_dict["soldPropertyData"]["property"]["images"]:
			#	house_image_urls.append(image['url'])
			#v["house_image_urls"] = house_image_urls
			# location of the house
			latitude = json_dict["soldPropertyData"]["property"]["location"]["latitude"]
			longitude = json_dict["soldPropertyData"]["property"]["location"]["longitude"]
			v["latitude"] = latitude
			v["longitude"] = longitude
			# proximity to the stations 
			station_names, distances = [], []
			for station in json_dict["soldPropertyData"]["property"]["nearestStations"]:
				station_names.append(station["name"])
				distances.append(station["distance"])
			station_proximities = {station:distance for station, distance in zip(station_names, distances)}
			v["station_proximities"] = station_proximities
			# size of the property
			units, sizes = [], []
			for i in json_dict["soldPropertyData"]["property"]["sizings"]:
				units.append(i['unit'])
				sizes.append(i['maximumSize'])
			dimensions = {unit:size for unit, size in zip(units, sizes)}
            # remove all of the size data which is not sqm or sgf
            for label, size in dimensions.items():
                if label not in m_values and label not in f_values:
                    del dimensions[label]
            
            # still haven't delt with the potential of having both a sqm and sqf, or just a sqf

			v["property_size"] = dimensions
            
            # JUST ABOVE HERE IS WHERE I COULD MAKE IT SO ONLY SQ FT IS OBTAINED - Still needs to be finished off

			# number of bedrooms and bathrooms
			bedrooms = json_dict["soldPropertyData"]["property"]['bedrooms']
			bathrooms = json_dict["soldPropertyData"]["property"]['bathrooms']
			v["bedrooms"] = bedrooms
			v["bathrooms"] = bathrooms

			# Whether the property is a new build or not
			new_build = json_dict["soldPropertyData"]["transactions"][0]['newBuild']
			v["new_build"] = new_build

In [30]:
# change price from a string into an int
def price_int(x):
    """Return an integer from a price string"""
    a = x[1:]
    b = a.replace(",", "")
    b = int(b)
    return b

# run the above function for all of the stored prices
for features in houses.values():
    y = features['price']
    z = price_int(y)
    features['price'] = z

In [53]:
def size_floorplan_1(floorplan):
    """Return the size data (in sq ft) from the floorplan"""
    reader = easyocr.Reader(['en']) # GPU=false taken out for now
    result = reader.readtext(floorplan)
    
    scraped_text = ''
    for i in result:
        scraped_text += i[1]
        scraped_text += ","
    text_list = scraped_text.split(",")
    
    m_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square meters|square metres|square meter|square metre|square m|square mt|sqmt|sq mt|sq.mt|sq. mt|sqm|sq m|sq.m|sq. m|sq: m|sq:m|sq :m|sq:  m|sq : m|meters2|metres2|meter2|metre2|mt2|m2)", re.IGNORECASE)
    f_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square feet|square ft|square f|sqft|sq ft|sq.ft|sq. ft|sqf|sq f|sq: f|sq:f|sq :f|sq:  f|sq : f|sq.f|sq. f|feet2|ft2|f2)", re.IGNORECASE)
    house_size = []
    for x in text_list:
        for match in re.finditer(m_patterns, x):
            #print('%s : %s' % (match.groups()))
            house_size.append(list(match.groups()))
        for match in re.finditer(f_patterns, x):
            #print('%s : %s' % (match.groups())) 
            house_size.append(list(match.groups()))
    for a in house_size:
        try:
            a[0] = int(a[0])
        except ValueError:
            a[0] = float(a[0])
            
    # then here you could find the
    house_size = dict((house_size))
    print(house_size)
    # Code to take the largest returned value from the floorplan
    amount = 0
    for size in house_size.keys():
        if size > amount:
            amount = size
        largest_size = {}
        largest_size[amount] = house_size[amount]
        largest_size[amount] = largest_size[amount].lower()
        
        
        # need to add values with colons, :
        # alternatively, you could just remove the colon. This would also nehate this issue

        metre_values = ["square meters", "square metres", "square meter", "square metre", "square m", "square mt", "sqmt", "sq mt",
                "sq.mt", "sq. mt", "sqm", "sq m", "sq.m", "sq. m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
        feet_values = ["square feet","square ft","square f","sqft","sq ft","sq.ft","sq. ft","sqf","sq f","sq.f","sq. f","feet2",
               "ft2","f2"]


        if largest_size[amount] in metre_values:
            # convert the size into square ft if sq m is scraped
            del largest_size[amount]
            largest_size[amount*10.764] = "sq ft"
        elif largest_size[amount] in feet_values:
            largest_size[amount] = "sq ft"
        
        # it would be useful to add a litte conversion here, if the scraped data is in sq m, then multiply be 10.764,
        
        return largest_size

In [111]:
def size_floorplan_2(floorplan):
    """Return the size data (in sq ft) from the floorplan"""
    reader = easyocr.Reader(['en']) # GPU=false taken out for now
    result = reader.readtext(floorplan)
    
    scraped_text = ''
    for i in result:
        scraped_text += i[1]
        scraped_text += ","
    text_list = scraped_text.split(",")
    
    m_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square meters|square metres|square meter|square metre|square m|square mt|sqmt|sq mt|sq.mt|sq. mt|sqm|sq m|sq.m|sq. m|sq: m|sq:m|sq :m|sq : m|sq :m|meters2|metres2|meter2|metre2|mt2|m2)", re.IGNORECASE)
    f_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square feet|square ft|square f|sqft|sq ft|sq.ft|sq. ft|sqf|sq f|sq.f|sq. f|sq: m|sq:m|sq :m|sq : m|sq :m|feet2|ft2|f2)", re.IGNORECASE)
    house_size = []
    for x in text_list:
        for match in re.finditer(m_patterns, x):
            #print('%s : %s' % (match.groups()))
            house_size.append(list(match.groups()))
        for match in re.finditer(f_patterns, x):
            #print('%s : %s' % (match.groups())) 
            house_size.append(list(match.groups()))
    for a in house_size:
        try:
            a[0] = int(a[0])
        except ValueError:
            a[0] = float(a[0])
            
    # then here you could find the
    house_size = dict((house_size))
    print(house_size)
    house_size_swap = {}
    # code to swap keys and values
    for k, v in house_size.items():
        house_size_swap[v] = k

    house_size = house_size_swap
    
    # Code to take the largest returned value from the floorplan numbers
    amount = max(house_size.values())
    for i in house_size.keys():
        if house_size[i] == amount:
            size_label = i.lower()
    
    # Remove punctuation from the size labels       
    if "," in size_label:
        size_label = size_label.replace(",", "")
    if ":" in size_label:
        size_label = size_label.replace(":", "")
    if "." in size_label:
        size_label = size_label.replace(".", "")
    
    largest_size = {size_label:amount}
    
    
            
    # need to add values with colons, :
    # alternatively, you could just remove the colon. This would also nehate this issue

    metre_values = ["square meters", "square metres", "square meter", "square metre", "square m", "square mt", "sqmt", "sq mt", "sq mt", "sq mt", "sqm", "sq m", "sq m", "sq m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
    feet_values = ["square feet","square ft","square f","sqft","sq ft","sq ft","sq. ft","sqf","sq f","sq f","sq f","feet2", "ft2","f2"]


    if size_label in metre_values:
        # convert the size into square ft if sq m is obtained
        largest_size["sq ft"] = amount*10.764
        del largest_size[size_label]
    elif size_label in feet_values:
        largest_size["sq ft"] = amount
        
        # it would be useful to add a litte conversion here, if the scraped data is in sq m, then multiply be 10.764,
        
    return largest_size

In [112]:
size_floorplan_2('https://media.rightmove.co.uk/14k/13345/115507229/13345_31062102_FLP_00_0000.jpeg')

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.7% Complete{206: 'Sq Ft', 9: 'Sq M', 97: 'Sq Ft', 121: 'Sq M', 1303: 'Sq Ft'}


{'sq ft': 1303}

In [34]:
outstanding_schools = [[51.4974948, -0.1356583],
 [51.5202607, -0.0293396],
 [51.4935082, -0.1178424],
 [51.6569225, -0.1949252],
 [51.5202607, -0.0293396],
 [51.5345448, -0.2043853],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.6569225, -0.1949252],
 [51.5906113, -0.1109709],
 [51.5202607, -0.0293396],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.5672808, -0.2710568],
 [51.6569225, -0.1949252],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.5202607, -0.0293396],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.4990805, -0.1938253],
 [51.5255162, 0.0352163],
 [51.5255162, 0.0352163],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.5202607, -0.0293396],
 [51.5436387, -0.0553621],
 [51.4935082, -0.1178424],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5886383, -0.0117625],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.5886121, 0.0823982],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.5886383, -0.0117625],
 [51.5132537, -0.3043136],
 [51.5906113, -0.1109709],
 [51.458373, -0.1891356],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5672808, -0.2710568],
 [51.5886383, -0.0117625],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.5202607, -0.0293396],
 [51.4609218, -0.373149],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.4933675, 0.0098214],
 [51.5886121, 0.0823982],
 [51.5886121, 0.0823982],
 [51.502781, -0.087738],
 [51.4935082, -0.1178424],
 [51.5540666, 0.134017],
 [51.4935082, -0.1178424],
 [51.5886121, 0.0823982],
 [51.5886383, -0.0117625],
 [51.4611509, -0.0073177],
 [51.5390261, -0.1425516],
 [51.4609218, -0.373149],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5132537, -0.3043136],
 [51.4990805, -0.1938253],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.5886121, 0.0823982],
 [51.5202607, -0.0293396],
 [51.5390261, -0.1425516],
 [51.502781, -0.087738],
 [51.4609218, -0.373149],
 [51.5436387, -0.0553621],
 [51.4974948, -0.1356583],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5123443, -0.0909852],
 [51.538621, -0.1028346],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.5255162, 0.0352163],
 [51.5886383, -0.0117625],
 [51.538621, -0.1028346],
 [51.4935082, -0.1178424],
 [51.4990156, -0.22915],
 [51.538621, -0.1028346],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.502781, -0.087738],
 [51.4935082, -0.1178424],
 [51.5202607, -0.0293396],
 [51.502781, -0.087738],
 [51.5202607, -0.0293396],
 [51.5540666, 0.134017],
 [51.4990156, -0.22915],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.5132537, -0.3043136],
 [51.6569225, -0.1949252],
 [51.4611509, -0.0073177],
 [51.4935082, -0.1178424],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.4933675, 0.0098214],
 [51.406025, 0.013156],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.5886383, -0.0117625],
 [51.502781, -0.087738],
 [51.4990805, -0.1938253],
 [51.4097742, -0.2108084],
 [51.4933675, 0.0098214],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.502781, -0.087738],
 [51.4974948, -0.1356583],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.4611509, -0.0073177],
 [51.6522994, -0.0807119],
 [51.5132537, -0.3043136],
 [51.6522994, -0.0807119],
 [51.5132537, -0.3043136],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.6522994, -0.0807119],
 [51.5436387, -0.0553621],
 [51.4611509, -0.0073177],
 [51.502781, -0.087738],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.5540666, 0.134017],
 [51.461311, -0.303742],
 [51.502781, -0.087738],
 [51.5672808, -0.2710568],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5886383, -0.0117625],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.577924, 0.2120829],
 [51.4990805, -0.1938253],
 [51.5886383, -0.0117625],
 [51.461311, -0.303742],
 [51.4935082, -0.1178424],
 [51.4990156, -0.22915],
 [51.5255162, 0.0352163],
 [51.502781, -0.087738],
 [51.5255162, 0.0352163],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.538621, -0.1028346],
 [51.3769529, -0.0956895],
 [51.6569225, -0.1949252],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.5255162, 0.0352163],
 [51.4990805, -0.1938253],
 [51.439933, 0.154327],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.3769529, -0.0956895],
 [51.5436387, -0.0553621],
 [51.439933, 0.154327],
 [51.6569225, -0.1949252],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.4097742, -0.2108084],
 [51.6522994, -0.0807119],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5886121, 0.0823982],
 [51.41233, -0.300689],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.502781, -0.087738],
 [51.580559, -0.341995],
 [51.439933, 0.154327],
 [51.5906113, -0.1109709],
 [51.5351832, -0.4481378],
 [51.5255162, 0.0352163],
 [51.4974948, -0.1356583],
 [51.4097742, -0.2108084],
 [51.5436387, -0.0553621],
 [51.5202607, -0.0293396],
 [51.5390261, -0.1425516],
 [51.6569225, -0.1949252],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.4935082, -0.1178424],
 [51.5540666, 0.134017],
 [51.4974948, -0.1356583],
 [51.4933675, 0.0098214],
 [51.5540666, 0.134017],
 [51.3769529, -0.0956895],
 [51.5351832, -0.4481378],
 [51.4990156, -0.22915],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177],
 [51.5886121, 0.0823982],
 [51.5906113, -0.1109709],
 [51.6522994, -0.0807119],
 [51.6522994, -0.0807119],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.4933675, 0.0098214],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.4097742, -0.2108084],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.502781, -0.087738],
 [51.577924, 0.2120829],
 [51.406025, 0.013156],
 [51.5906113, -0.1109709],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5132537, -0.3043136],
 [51.4611509, -0.0073177],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.4990805, -0.1938253],
 [51.6522994, -0.0807119],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.4935082, -0.1178424],
 [51.5906113, -0.1109709],
 [51.41233, -0.300689],
 [51.5906113, -0.1109709],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4609218, -0.373149],
 [51.4611509, -0.0073177],
 [51.5906113, -0.1109709],
 [51.580559, -0.341995],
 [51.3769529, -0.0956895],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.5886121, 0.0823982],
 [51.461311, -0.303742],
 [51.4935082, -0.1178424],
 [51.5132537, -0.3043136],
 [51.6569225, -0.1949252],
 [51.4097742, -0.2108084],
 [51.5906113, -0.1109709],
 [51.6569225, -0.1949252],
 [51.5886383, -0.0117625],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4609218, -0.373149],
 [51.577924, 0.2120829],
 [51.5351832, -0.4481378],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.439933, 0.154327],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.5351832, -0.4481378],
 [51.4609218, -0.373149],
 [51.5540666, 0.134017],
 [51.6569225, -0.1949252],
 [51.3769529, -0.0956895],
 [51.439933, 0.154327],
 [51.6569225, -0.1949252],
 [51.4609218, -0.373149],
 [51.4990805, -0.1938253],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.3769529, -0.0956895],
 [51.4990156, -0.22915],
 [51.5351832, -0.4481378],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.4609218, -0.373149],
 [51.5436387, -0.0553621],
 [51.5390261, -0.1425516],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.4974948, -0.1356583],
 [51.5390261, -0.1425516],
 [51.458373, -0.1891356],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.4974948, -0.1356583],
 [51.6522994, -0.0807119],
 [51.4990156, -0.22915],
 [51.4990805, -0.1938253],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5886383, -0.0117625],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.5540666, 0.134017],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4097742, -0.2108084],
 [51.5390261, -0.1425516],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5123443, -0.0909852],
 [51.461311, -0.303742],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.538621, -0.1028346],
 [51.5886121, 0.0823982],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.6569225, -0.1949252],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.6569225, -0.1949252],
 [51.4935082, -0.1178424],
 [51.580559, -0.341995],
 [51.4935082, -0.1178424],
 [51.538621, -0.1028346],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.5255162, 0.0352163],
 [51.4990805, -0.1938253],
 [51.4974948, -0.1356583],
 [51.4990156, -0.22915],
 [51.5672808, -0.2710568],
 [51.6522994, -0.0807119],
 [51.538621, -0.1028346],
 [51.5906113, -0.1109709],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.5132537, -0.3043136],
 [51.5202607, -0.0293396],
 [51.5132537, -0.3043136],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5351832, -0.4481378],
 [51.5390261, -0.1425516],
 [51.580559, -0.341995],
 [51.458373, -0.1891356],
 [51.538621, -0.1028346],
 [51.4935082, -0.1178424],
 [51.580559, -0.341995],
 [51.5351832, -0.4481378],
 [51.502781, -0.087738],
 [51.4990156, -0.22915],
 [51.5886121, 0.0823982],
 [51.6569225, -0.1949252],
 [51.4990805, -0.1938253],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.5351832, -0.4481378],
 [51.4990805, -0.1938253],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.6522994, -0.0807119],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.4611509, -0.0073177],
 [51.580559, -0.341995],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.6569225, -0.1949252],
 [51.458373, -0.1891356],
 [51.502781, -0.087738],
 [51.41233, -0.300689],
 [51.5540666, 0.134017],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5351832, -0.4481378],
 [51.6522994, -0.0807119],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.502781, -0.087738],
 [51.5132537, -0.3043136],
 [51.5351832, -0.4481378],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.5672808, -0.2710568],
 [51.3769529, -0.0956895],
 [51.4990805, -0.1938253],
 [51.4990156, -0.22915],
 [51.4974948, -0.1356583],
 [51.5886121, 0.0823982],
 [51.461311, -0.303742],
 [51.5351832, -0.4481378],
 [51.5390261, -0.1425516],
 [51.5255162, 0.0352163],
 [51.538621, -0.1028346],
 [51.4097742, -0.2108084],
 [51.5886121, 0.0823982],
 [51.3769529, -0.0956895],
 [51.5390261, -0.1425516],
 [51.577924, 0.2120829],
 [51.5886383, -0.0117625],
 [51.406025, 0.013156],
 [51.4990805, -0.1938253],
 [51.3769529, -0.0956895],
 [51.6522994, -0.0807119],
 [51.4935082, -0.1178424],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.406025, 0.013156],
 [51.6522994, -0.0807119],
 [51.4990156, -0.22915],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.4974948, -0.1356583],
 [51.6522994, -0.0807119],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5886383, -0.0117625],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.5672808, -0.2710568],
 [51.5672808, -0.2710568],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.538621, -0.1028346],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.580559, -0.341995],
 [51.5390261, -0.1425516],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.5906113, -0.1109709],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.5390261, -0.1425516],
 [51.458373, -0.1891356],
 [51.580559, -0.341995],
 [51.5255162, 0.0352163],
 [51.5906113, -0.1109709],
 [51.4990156, -0.22915],
 [51.4990805, -0.1938253],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4611509, -0.0073177],
 [51.3769529, -0.0956895],
 [51.4933675, 0.0098214],
 [51.577924, 0.2120829],
 [51.6522994, -0.0807119],
 [51.458373, -0.1891356],
 [51.4933675, 0.0098214],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.6522994, -0.0807119],
 [51.4933675, 0.0098214],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.5351832, -0.4481378],
 [51.5436387, -0.0553621],
 [51.4990805, -0.1938253],
 [51.6569225, -0.1949252],
 [51.461311, -0.303742],
 [51.6569225, -0.1949252],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4990805, -0.1938253],
 [51.5886121, 0.0823982],
 [51.5351832, -0.4481378],
 [51.6522994, -0.0807119],
 [51.5255162, 0.0352163],
 [51.461311, -0.303742],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.5436387, -0.0553621],
 [51.5906113, -0.1109709],
 [51.580559, -0.341995],
 [51.5202607, -0.0293396],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.5436387, -0.0553621],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.6569225, -0.1949252],
 [51.4933675, 0.0098214],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.5132537, -0.3043136],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.580559, -0.341995],
 [51.439933, 0.154327],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.580559, -0.341995],
 [51.4935082, -0.1178424],
 [51.461311, -0.303742],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.4935082, -0.1178424],
 [51.458373, -0.1891356],
 [51.439933, 0.154327],
 [51.580559, -0.341995],
 [51.4933675, 0.0098214],
 [51.461311, -0.303742],
 [51.3769529, -0.0956895],
 [51.577924, 0.2120829],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.5132537, -0.3043136],
 [51.4974948, -0.1356583],
 [51.5886383, -0.0117625],
 [51.4990156, -0.22915],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.3769529, -0.0956895],
 [51.538621, -0.1028346],
 [51.5436387, -0.0553621],
 [51.538621, -0.1028346],
 [51.439933, 0.154327],
 [51.458373, -0.1891356],
 [51.3769529, -0.0956895],
 [51.5672808, -0.2710568],
 [51.4935082, -0.1178424],
 [51.41233, -0.300689],
 [51.577924, 0.2120829],
 [51.6569225, -0.1949252],
 [51.5202607, -0.0293396],
 [51.6569225, -0.1949252],
 [51.41233, -0.300689],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.4609218, -0.373149],
 [51.5886121, 0.0823982],
 [51.458373, -0.1891356],
 [51.5132537, -0.3043136],
 [51.458373, -0.1891356],
 [51.4611509, -0.0073177],
 [51.461311, -0.303742],
 [51.4933675, 0.0098214],
 [51.4097742, -0.2108084],
 [51.5436387, -0.0553621],
 [51.577924, 0.2120829],
 [51.4990156, -0.22915],
 [51.406025, 0.013156],
 [51.580559, -0.341995],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5123443, -0.0909852],
 [51.4990156, -0.22915],
 [51.5906113, -0.1109709],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.3769529, -0.0956895],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.5886121, 0.0823982],
 [51.4609218, -0.373149],
 [51.3769529, -0.0956895],
 [51.538621, -0.1028346],
 [51.6569225, -0.1949252],
 [51.5351832, -0.4481378],
 [51.6569225, -0.1949252],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.4935082, -0.1178424],
 [51.4974948, -0.1356583],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5351832, -0.4481378],
 [51.5540666, 0.134017],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.458373, -0.1891356],
 [51.5672808, -0.2710568],
 [51.458373, -0.1891356],
 [51.4974948, -0.1356583],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.5540666, 0.134017],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5390261, -0.1425516],
 [51.4609218, -0.373149],
 [51.4990156, -0.22915],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.502781, -0.087738],
 [51.461311, -0.303742],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.5886121, 0.0823982],
 [51.580559, -0.341995],
 [51.4097742, -0.2108084],
 [51.5886383, -0.0117625],
 [51.4611509, -0.0073177],
 [51.577924, 0.2120829],
 [51.5132537, -0.3043136],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.502781, -0.087738],
 [51.538621, -0.1028346],
 [51.4097742, -0.2108084],
 [51.5132537, -0.3043136],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.6569225, -0.1949252],
 [51.41233, -0.300689],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.458373, -0.1891356],
 [51.5202607, -0.0293396],
 [51.502781, -0.087738],
 [51.577924, 0.2120829],
 [51.5906113, -0.1109709],
 [51.4609218, -0.373149],
 [51.4609218, -0.373149],
 [51.5540666, 0.134017],
 [51.577924, 0.2120829],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.4933675, 0.0098214],
 [51.5886383, -0.0117625],
 [51.406025, 0.013156],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.5202607, -0.0293396],
 [51.461311, -0.303742],
 [51.4974948, -0.1356583],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.458373, -0.1891356],
 [51.5886121, 0.0823982],
 [51.4097742, -0.2108084],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.4611509, -0.0073177],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.4935082, -0.1178424],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4990805, -0.1938253],
 [51.6569225, -0.1949252],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.5132537, -0.3043136],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.3769529, -0.0956895],
 [51.5390261, -0.1425516],
 [51.4990156, -0.22915],
 [51.3769529, -0.0956895],
 [51.5886121, 0.0823982],
 [51.406025, 0.013156],
 [51.5202607, -0.0293396],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.6569225, -0.1949252],
 [51.5390261, -0.1425516],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.538621, -0.1028346],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5886121, 0.0823982],
 [51.6569225, -0.1949252],
 [51.580559, -0.341995],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.4990156, -0.22915],
 [51.5351832, -0.4481378],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.5255162, 0.0352163],
 [51.538621, -0.1028346],
 [51.4609218, -0.373149],
 [51.5390261, -0.1425516],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.4990156, -0.22915],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.5390261, -0.1425516],
 [51.461311, -0.303742],
 [51.5132537, -0.3043136],
 [51.461311, -0.303742],
 [51.3769529, -0.0956895],
 [51.5540666, 0.134017],
 [51.4933675, 0.0098214],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5255162, 0.0352163],
 [51.580559, -0.341995],
 [51.5886121, 0.0823982],
 [51.458373, -0.1891356],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.5672808, -0.2710568],
 [51.538621, -0.1028346],
 [51.5540666, 0.134017],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.5132537, -0.3043136],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4933675, 0.0098214],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.461311, -0.303742],
 [51.538621, -0.1028346],
 [51.4990156, -0.22915],
 [51.4935082, -0.1178424],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.41233, -0.300689],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.5132537, -0.3043136],
 [51.538621, -0.1028346],
 [51.41233, -0.300689],
 [51.538621, -0.1028346],
 [51.4990805, -0.1938253],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.4974948, -0.1356583],
 [51.4935082, -0.1178424],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.4974948, -0.1356583],
 [51.538621, -0.1028346],
 [51.538621, -0.1028346],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.4935082, -0.1178424],
 [51.4933675, 0.0098214],
 [51.3769529, -0.0956895],
 [51.4990156, -0.22915],
 [51.5390261, -0.1425516],
 [51.4097742, -0.2108084],
 [51.4990805, -0.1938253],
 [51.4097742, -0.2108084],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.577924, 0.2120829],
 [51.4974948, -0.1356583],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.4609218, -0.373149],
 [51.5255162, 0.0352163],
 [51.580559, -0.341995],
 [51.5886383, -0.0117625],
 [51.4990805, -0.1938253],
 [51.502781, -0.087738],
 [51.3769529, -0.0956895],
 [51.577924, 0.2120829],
 [51.577924, 0.2120829],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.538621, -0.1028346],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.4990805, -0.1938253],
 [51.4935082, -0.1178424],
 [51.406025, 0.013156],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.6569225, -0.1949252],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.502781, -0.087738],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.5886121, 0.0823982],
 [51.577924, 0.2120829],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177]]

In [50]:
def nearest_outstanding_school(lat, lon):
    """Return the KM distance to the nearest outstanding school"""
    coords_1 = (lat, lon)
    lower_boundary, upper_boundary = -0.05, 0.05
    school_distances = []
    for i in outstanding_schools:
        if (i[0] - lat > upper_boundary) or i[0] - lat < lower_boundary:
            continue
        elif (i[1] - lon > upper_boundary) or i[1] - lon < lower_boundary:
            continue
        else:
            coords_2 = (i[0], i[1])
            school_distances.append(geopy.distance.geodesic(coords_1, coords_2).km)

    return min(school_distances)

In [55]:
# loop through all of the houses, if they don't have a property size but do have a floorplan, apply the OCR, and then return 
# value (note a nonetype may be returned so you'll have to deal with that). 
# If they have a don't have a fl

# Loop through all of the houses, if they have their lon and lat available, return the distance to the nearest 
# outstanding school
for y in houses.values():
    # Generate the feature of distance to the nearest outstanding school
    if 'latitude' in y.keys() and 'longitude' in y.keys():
        distance_km = nearest_outstanding_school(y['latitude'], y['longitude'])
        y['nearest_outstanding_school'] = distance_km
    
    # Attempt to generate the feature of the sq ft of the property
    if (y['property_size'] == {}) and ('floorplan_url' in y.keys()):
        try:
            ocr_size = size_floorplan_2(y['floorplan_url'])
        except Exception:
            y['property_size'] = ''
        else:
            if ocr_size == None:
                y['property_size'] = ''
            y['property_size'] = ocr_size
    # If a property_size dictionary already exists, use it to find the sq ft of the property 
    # code that handles when the property data has a dictionary with sqf and/or sq/f values in it
    # Now you just need the f_values and m_values lists in the block of code
    elif len(y['property_size']) >= 1:
        # loop through the this dictionary
        # if the dictionary contains a sq m value, set the value of property size to its sq f equivalent, 
        # if the dictionary contains a sq f value, set it
        for i in y['property_size'].keys():
            p = i.strip().lower()
            if p in f_values:
                y['property_size'] = y['property_size'][i]
            elif p in m_values:
                y['property_size'] = y['property_size'][i]*10.764
    
    # This is where you could do an else statement, and set all of the prop_size dictionaries which end up in the data set 
    # as ''
        

            
            
        
 # need to handle for the situation when there is data in the property size dict, but its not related to sq sizes, 
# and there is a floorplan URL
        
    
#     else:
#         if 'floorplan_url' in y.keys():
#             ocr_size = size_floorplan_2(y['floorplan_url'])
#             if ocr_size == None:
#                 y['property_size'] = ''
#             y['property_size'] = ocr_size
    
        

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.7% Complete

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |███████████████████████████████████████████████████| 103.6% Complete

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


...

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |███████████████████████████████████████████████████| 103.1% Complete

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


{}
Progress: |████████████████████████████████████████████████████████████████| 128.9% Complete

ValueError: Could not find a backend to open `C:\Users\jacks\AppData\Local\Temp\tmpyfx7wbfq`` with iomode `ri`.

In [56]:
houses

{'105, Canonsleigh Road, Dagenham, Greater London RM9 4DH': {'property_type': 'Terraced',
  'price': 390000,
  'date': '16 May 2022',
  'url': ''},
 '59, Bateman Close, Barking, Greater London IG11 8QS': {'property_type': 'Flat',
  'price': 225000,
  'date': '12 May 2022',
  'url': 'https://www.rightmove.co.uk/house-prices/details/england-123105338-14974252?s=791fe9e857a8dea92f2d04177af0a010ed90647d8449404cd8f3099af0e6d49f',
  'features': [],
  'floorplan_url': 'https://media.rightmove.co.uk/60k/59646/123105338/59646_CW0434_FLP_00_0000.jpeg',
  'latitude': 51.54126,
  'longitude': 0.08243,
  'station_proximities': {'Barking Station': 0.14006575393753187,
   'Upney Station': 0.8457992920654402,
   'Ilford Station': 1.3208233936340128},
  'property_size': {'ac': 0.01, 'sqft': 506, 'sqm': 47, 'ha': 0},
  'bedrooms': 1,
  'bathrooms': 1,
  'new_build': False,
  'nearest_outstanding_school': 3.7150173193710945},
 '54, Greenslade Road, Barking, Greater London IG11 9XE': {'property_type': 'Fl

In [82]:
# Need to convert the data into a quantitative format, which regression algorithms can learn/predict from
properties = {}
index = 0
# loop through all of the 
for h, i in houses.items():
    house_data = []
    if i['property_type'] == 'Flat':
        prop_type = 1
    elif i['property_type'] == 'Terraced':
        prop_type == 2
    elif i['property_type'] == 'Semi-Detached':
        prop_type = 3 
    elif i['property_type'] == 'Detached':
        prop_type = 4
    else:
        prop_type = 5
    house_data.append(prop_type)
    
    # Convert the data into the necessary string format
    date_obj = datetime.datetime.strptime(i['date'], "%d %b %Y")
    date_str = date_obj.strftime("%Y-%m-%d")
    house_data.append(date_str)
    
    if 'station_proximities' in i.keys() and i['station_proximities'] != {}:
        station_distance = min(i['station_proximities'].values())
        house_data.append(station_distance)
    else:
        house_data.append("NA")
        
    # You should have handled for the property_size difficulties by now
    if i['property_size'] == {} or i['property_size'] == '' or i["property_size"] == None:
        house_data.append("NA")
    else:
        # This should return the sq foot of the property
        house_data.append(i["property_size"])
    
    # Add the bedrooms and bathrooms data   
    if "bedrooms" not in i.keys():
        house_data.append("NA")
    else:
        house_data.append(i["bedrooms"])
        
    if 'bathrooms' not in i.keys():
        house_data.append("NA")
    else:
        house_data.append(i["bathrooms"])
        
    # New build
    if 'new_build' not in i.keys():
        house_data.append("NA")
    else:
        if i["new_build"]:
            house_data.append(1)
        else:
            house_data.append(0)
    
    if 'nearest_outstanding_school' not in i.keys():
        house_data.append("NA")
    else:
        house_data.append(i["nearest_outstanding_school"])
        
    house_data.append(i["price"])
    
    

            
     
        
        
    properties[h] = house_data
    index += 1
    
    

In [83]:
properties

{'105, Canonsleigh Road, Dagenham, Greater London RM9 4DH': [1, '2022-05-16'],
 '59, Bateman Close, Barking, Greater London IG11 8QS': [1, '2022-05-12'],
 '54, Greenslade Road, Barking, Greater London IG11 9XE': [1, '2022-05-11'],
 '21, Felton Road, Barking, Greater London IG11 7XZ': [1, '2022-05-10'],
 '28, Sparsholt Road, Barking, Greater London IG11 7YQ': [1, '2022-04-29'],
 '36, Halsham Crescent, Barking, Greater London IG11 9HG': [1, '2022-04-14'],
 '10a, Sandringham Road, Barking, Greater London IG11 9AA': [3, '2022-04-14'],
 '60, Wedderburn Road, Barking, Greater London IG11 7XG': [3, '2022-04-14'],
 '33, Westminster Gardens, Barking, Greater London IG11 0BJ': [3,
  '2022-04-14'],
 '164, Park Avenue, Barking, Greater London IG11 8QY': [3, '2022-04-08'],
 '29, Edgefield Avenue, Barking, Greater London IG11 9JL': [3, '2022-04-08'],
 '94, Bastable Avenue, Barking, Greater London IG11 0LF': [1, '2022-04-08'],
 '127, Fanshawe Avenue, Barking, Greater London IG11 8RF': [1, '2022-04-08

Now ready to put the data into a pandas dataframe, and to use scikit learn to create some ML models

In [ ]:
# Property type -> 
# Flat 1, Terranced 2, Semi-Detached 3, Detached 4
# Date (get it into YYYY-MM-DD format)
# Smallest station proximity
# property_size (in sqft), may need to do some conversions
# bedrooms
# bathrooms
# newbuild, 0 if false
# nearest outstanding school distance

# price (label)

In [ ]:
# Approach to help find the prices of nearby, similar properties
# THIS CODE NEEDS TO BE PERFORMED AFTER ALL OF THE HOUSES ARE STORED
# or even you could it after each URL iteration of the local area URL's, to save looping through all of the houses

######################## DO NOT RUN THIS CODE ###########################


for i, v in houses.items():
    local_similar_prices = []
    # need to store the postcode, each house has a postcode so this will be easier
    postcode = i[-8:]
    if postcode[0] == " ":
        postcode = i[-7:]
    postcode_area = postcode.split(" ")[0]
    prop_type = v["property_type"]
    
    for a, b in houses.items():
        postcode = i[-8:]
        if postcode[0] == " ":
            postcode = i[-7:]
        postcode_area1 = postcode.split(" ")[0]
        if postcode_area == postcode_area1:
            if b["property_type"] == prop_type:
                local_similar_prices.append(b['price'])
    
    avg_local_price = avg(local_similar_prices)
    v["avg_local_price"] = avg_local_price
    
    
####################################################################

dict_keys(['hi', 'hello'])

False

{'results': {'title': 'House Prices in Beacontree Heath',
  'metaTagDescription': 'The average price for a property in Beacontree Heath is £364,885 over the last year. Use Rightmove online house price checker tool to find out exactly how much properties sold for in Beacontree Heath since 1995 (based on official Land Registry data).',
  'containsScotland': False,
  'resultCount': '2,074',
  'properties': [{'address': '143, Oxlow Lane, Dagenham, Greater London RM10 7ST',
    'propertyType': 'Terraced',
    'bedrooms': 3,
    'images': {'imageUrl': 'https://media.rightmove.co.uk/dir/14k/13289/13289_JF02060341S_IMG_01_max_135x100.jpg',
     'count': 8},
    'hasFloorPlan': False,
    'transactions': [{'displayPrice': '£380,000',
      'dateSold': '7 Apr 2022',
      'tenure': 'Freehold',
      'newBuild': False},
     {'displayPrice': '£198,000',
      'dateSold': '31 Aug 2007',
      'tenure': 'Freehold',
      'newBuild': False}],
    'location': {'lat': 51.55104, 'lng': 0.14865},
    'd

In [121]:
b = {'hi':True, "Hye":3}

In [122]:
if b['hi']:
    print("yo")

yo


In [101]:
d = "hey,"

In [102]:
d.replace(",", ".")

'hey.'

In [103]:
d

'hey,'

In [118]:
e = " hEy"

In [120]:
e.strip().lower()

'hey'